# 1. Import Python Libraries

In [269]:
import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd
from base import ROOT_DIR
import geopandas as gpd

In [270]:
%config Completer.use_jedi = False

## 1.1. Set up the path

In [271]:
eaglebine_dir = ROOT_DIR / "gtx/data/Eaglebine/Eaglebine"
duvernay_dir = ROOT_DIR / "gtx/data/Duvernay/Duvernay"

# 2. Load Eaglebine Temperature Data

## 2.1 Load BHT Data

In [272]:
egb_bht = pd.read_excel(eaglebine_dir / 'Eaglebine BHT TSC data for SPE April 21 2020.xlsx')
egb_bht['TempC_BHT'] = (egb_bht['BHTorMRT (maximum recorded temperature) oF'] - 32) * (5./9.)
egb_bht.head()

,UWI,SurfLat,SurfLong,TD (ft),GL(ft),BHT_below sea level (ft),BHT_ subsurface (ft),BHTorMRT (maximum recorded temperature) oF,TSC or ORT (time since circulation or original recorded time in hours),TempC_BHT
0,42013301410000,28.690426,-98.470138,19629.0,345.90,12222.50,12568.4,143.0,5.50,61.666667
1,42013301930000,28.779921,-98.208313,10775.0,397.40,10396.00,10793.4,225.0,11.00,107.222222
2,42013302760000,28.994360,-98.421799,6168.0,460.40,5733.00,6193.4,150.0,5.50,65.555556
3,42013305480000,28.759118,-98.156410,8198.0,410.40,7806.00,8216.4,170.0,5.00,76.666667
4,42013310190000,28.817400,-98.155319,11000.0,508.92,10509.48,11018.4,210.0,7.75,98.888889


In [273]:
# Transforms sstvd from ft to meters
egb_bht['BHT_below sea level (ft)'] = egb_bht['BHT_below sea level (ft)'] * 0.3048

## 2.2 Read in Eaglebine synthetic "true formation temperature"

In [274]:
egb_tt = pd.read_excel(eaglebine_dir / 'Eaglebine TrueTemp_Train2.xlsx')
egb_tt.head()

,UWI,Depth sub-sea (feet),True Temperature (oF)
0,42013301410000,-350,70.33
1,42013301410000,-300,70.33
2,42013301410000,-250,70.33
3,42013301410000,-200,71.49
4,42013301410000,-150,72.66


In [275]:
# Transforms sstvd from ft to meters
egb_tt['Depth sub-sea (feet)'] = egb_tt['Depth sub-sea (feet)'] * 0.3048

In [276]:
# convert to Celsius
egb_tt['TempTrue_degC'] = (egb_tt['True Temperature   (oF)'] - 32) * (5./9.)

## 2.3 Combine measured BHT and true temperature

In [277]:
egb_comb = egb_bht.merge(egb_tt, on='UWI', how='left')

In [278]:
# only keep from the synthetic data, the temperature at the elevation closest to the model
egb_comb['diff_depth'] = egb_comb['Depth sub-sea (feet)']-egb_comb['BHT_below sea level (ft)']
egb_comb['diff_depth_abs'] = np.abs(egb_comb['diff_depth'])
idx = egb_comb.groupby(['UWI'])['diff_depth_abs'].transform(min) == egb_comb['diff_depth_abs']
TrueTempUWI = egb_comb.loc[idx, ['UWI', 'diff_depth_abs', 'TempTrue_degC']]
TrueTempUWI = TrueTempUWI.copy(deep=True)
egb_comb = egb_bht.merge(TrueTempUWI, on='UWI', how='left')
egb_comb.head()

,UWI,SurfLat,SurfLong,TD (ft),GL(ft),BHT_below sea level (ft),BHT_ subsurface (ft),BHTorMRT (maximum recorded temperature) oF,TSC or ORT (time since circulation or original recorded time in hours),TempC_BHT,diff_depth_abs,TempTrue_degC
0,42013301410000,28.690426,-98.470138,19629.0,345.90,3725.418000,12568.4,143.0,5.50,61.666667,6.858000,141.555556
1,42013301930000,28.779921,-98.208313,10775.0,397.40,3168.700800,10793.4,225.0,11.00,107.222222,1.219200,132.038889
2,42013302760000,28.994360,-98.421799,6168.0,460.40,1747.418400,6193.4,150.0,5.50,65.555556,5.181600,89.161111
3,42013305480000,28.759118,-98.156410,8198.0,410.40,2379.268800,8216.4,170.0,5.00,76.666667,1.828800,107.961111
4,42013310190000,28.817400,-98.155319,11000.0,508.92,3203.289504,11018.4,210.0,7.75,98.888889,2.889504,133.722222


## 2.4 Read in static temperature information and merge into dataset

In [279]:
static_temp = pd.read_csv(ROOT_DIR / 'gtx/data/Data_static_logs.csv')
static_temp.head()

,Well_ID,Depth (ft),Temp (degC),Field
0,42013301930000,10700.0,132.778000,Eaglebine
1,42041305800000,7360.0,82.777778,Eaglebine
2,42041311700000,7828.0,118.888889,Eaglebine
3,42041311980000,7954.0,105.000000,Eaglebine
4,42051305380000,8380.0,111.111111,Eaglebine


In [280]:
egb_comb['UWI'] = egb_comb['UWI'].astype(str)
egb_comb = egb_comb.copy(deep=True)
egb_comb['TrueTemp_datasource_syn'] = 'synthetic'
static_temp['TrueTemp_datasource_stat'] = 'static_temp_logs'
egb_comb_final = egb_comb.merge(static_temp, left_on='UWI',right_on='Well_ID', how='left')

In [281]:
# Coalesce columns together with priority for true temperature measurements
egb_comb_final['TempC_Fin'] = egb_comb_final['Temp (degC)'].fillna(egb_comb_final['TempTrue_degC'])
egb_comb_final['TrueTemp_datasource'] = egb_comb_final['TrueTemp_datasource_stat'].\
fillna(egb_comb_final['TrueTemp_datasource_syn'])
egb_comb_final.head()

,UWI,SurfLat,SurfLong,TD (ft),GL(ft),BHT_below sea level (ft),BHT_ subsurface (ft),BHTorMRT (maximum recorded temperature) oF,TSC or ORT (time since circulation or original recorded time in hours),TempC_BHT,diff_depth_abs,TempTrue_degC,TrueTemp_datasource_syn,Well_ID,Depth (ft),Temp (degC),Field,TrueTemp_datasource_stat,TempC_Fin,TrueTemp_datasource
0,42013301410000,28.690426,-98.470138,19629.0,345.90,3725.418000,12568.4,143.0,5.50,61.666667,6.858000,141.555556,synthetic,NaN,NaN,NaN,NaN,NaN,141.555556,synthetic
1,42013301930000,28.779921,-98.208313,10775.0,397.40,3168.700800,10793.4,225.0,11.00,107.222222,1.219200,132.038889,synthetic,42013301930000,10700.0,132.778,Eaglebine,static_temp_logs,132.778000,static_temp_logs
2,42013302760000,28.994360,-98.421799,6168.0,460.40,1747.418400,6193.4,150.0,5.50,65.555556,5.181600,89.161111,synthetic,NaN,NaN,NaN,NaN,NaN,89.161111,synthetic
3,42013305480000,28.759118,-98.156410,8198.0,410.40,2379.268800,8216.4,170.0,5.00,76.666667,1.828800,107.961111,synthetic,NaN,NaN,NaN,NaN,NaN,107.961111,synthetic
4,42013310190000,28.817400,-98.155319,11000.0,508.92,3203.289504,11018.4,210.0,7.75,98.888889,2.889504,133.722222,synthetic,NaN,NaN,NaN,NaN,NaN,133.722222,synthetic


# 3. Load Duvernay Temperature Data

## 3.1 Read BHT data and "true temp" data

In [282]:
duv_tt = pd.read_excel(duvernay_dir / 'Duvenay TrueTemp_Train.xlsx')
duv_bht = pd.read_excel(duvernay_dir / 'Duvernay DST BHT for SPE April 20 2021.xlsx')
duv_tt.head()

,UWI,Depths subsea (m),True Temperature (oC)
0,100010107020W500,-720,2.53
1,100010107020W500,-700,2.54
2,100010107020W500,-680,2.54
3,100010107020W500,-660,3.30
4,100010107020W500,-640,4.06


In [283]:
# add in an extra column calculating the depth sub sea (elevation-depth)*-1 
duv_bht['Depth_SS(m)']=-1*(duv_bht['elevation M above sea level']-(duv_bht['DST Start Depth (MD) (m)']+
                                                                   duv_bht['DST End Depth (MD) (m)'])/2)
duv_bht.head()

,Well ID,DST Start Depth (MD) (m),DST End Depth (MD) (m),DST Bottom Hole Temp. (degC),DST Test Date,Test Type,DST Misrun,DST Number,Formation DSTd,Unnamed: 9,elevation M above sea level,UWI,Depth_SS(m)
0,100010107020W500,1410.6,1423.4,48.33,26025,DST,N,2,TRmontney,NaN,701.0,100010107020W500,716.0
1,100010608109W500,1686.0,1694.0,34.26,33647,DST,N,1,Dmuskeg,NaN,645.6,100010608109W500,1044.4
2,100010904012W500,3698.0,3718.0,104.40,30798,LRT,N,1,Melkton,NaN,1283.8,100010904012W500,2424.2
3,100011206021W400,822.0,825.0,29.00,29543,DST,N,4,Dwintrbrn,NaN,651.6,100011206021W400,171.9
4,100011506915W500,1801.0,1818.0,64.00,30760,DST,N,4,Dwabamun,NaN,873.1,100011506915W500,936.4


In [284]:
# merge
duv_comb = duv_bht.merge(duv_tt, on='UWI', how='left')
duv_comb.head()

,Well ID,DST Start Depth (MD) (m),DST End Depth (MD) (m),DST Bottom Hole Temp. (degC),DST Test Date,Test Type,DST Misrun,DST Number,Formation DSTd,Unnamed: 9,elevation M above sea level,UWI,Depth_SS(m),Depths subsea (m),True Temperature (oC)
0,100010107020W500,1410.6,1423.4,48.33,26025,DST,N,2,TRmontney,NaN,701.0,100010107020W500,716.0,-720.0,2.53
1,100010107020W500,1410.6,1423.4,48.33,26025,DST,N,2,TRmontney,NaN,701.0,100010107020W500,716.0,-700.0,2.54
2,100010107020W500,1410.6,1423.4,48.33,26025,DST,N,2,TRmontney,NaN,701.0,100010107020W500,716.0,-680.0,2.54
3,100010107020W500,1410.6,1423.4,48.33,26025,DST,N,2,TRmontney,NaN,701.0,100010107020W500,716.0,-660.0,3.30
4,100010107020W500,1410.6,1423.4,48.33,26025,DST,N,2,TRmontney,NaN,701.0,100010107020W500,716.0,-640.0,4.06


In [285]:
# keep only the synthetic model temperature data for the relevant depths for which there is BHT measurement
duv_comb['diff_depth'] = duv_comb['Depth_SS(m)']-duv_comb['Depths subsea (m)']
duv_comb['diff_depth_abs'] = np.abs(duv_comb['diff_depth'])
idx = duv_comb.groupby(['UWI'])['diff_depth_abs'].transform(min) == duv_comb['diff_depth_abs']

TrueTempUWI = duv_comb.loc[idx, ['UWI', 'diff_depth_abs', 'True Temperature (oC)']]
TrueTempUWI = TrueTempUWI.copy(deep=True)
duv_comb = duv_bht.merge(TrueTempUWI, on='UWI', how='left')
duv_comb = duv_comb.drop_duplicates(['UWI'])
duv_comb.head()

,Well ID,DST Start Depth (MD) (m),DST End Depth (MD) (m),DST Bottom Hole Temp. (degC),DST Test Date,Test Type,DST Misrun,DST Number,Formation DSTd,Unnamed: 9,elevation M above sea level,UWI,Depth_SS(m),diff_depth_abs,True Temperature (oC)
0,100010107020W500,1410.6,1423.4,48.33,26025,DST,N,2,TRmontney,NaN,701.0,100010107020W500,716.0,4.0,60.97
1,100010608109W500,1686.0,1694.0,34.26,33647,DST,N,1,Dmuskeg,NaN,645.6,100010608109W500,1044.4,4.4,63.98
2,100010904012W500,3698.0,3718.0,104.40,30798,LRT,N,1,Melkton,NaN,1283.8,100010904012W500,2424.2,NaN,NaN
3,100011206021W400,822.0,825.0,29.00,29543,DST,N,4,Dwintrbrn,NaN,651.6,100011206021W400,171.9,8.1,39.07
4,100011506915W500,1801.0,1818.0,64.00,30760,DST,N,4,Dwabamun,NaN,873.1,100011506915W500,936.4,3.6,69.47


## 3.2 Merge in static temperature log data

In [286]:
duv_comb['UWI'] = duv_comb['UWI'].astype(str)
duv_comb = duv_comb.copy(deep=True)
duv_comb['TrueTemp_datasource_syn'] = 'synthetic'
static_temp['TrueTemp_datasource_stat'] = 'static_temp_logs'
duv_comb_final = duv_comb.merge(static_temp, left_on='UWI',right_on='Well_ID', how='left')

In [287]:
# Coalesce columns together with priority for true temperature measurements
duv_comb_final['TempC_Fin'] = duv_comb_final['Temp (degC)'].fillna(duv_comb_final['True Temperature (oC)'])
duv_comb_final['TrueTemp_datasource'] = duv_comb_final['TrueTemp_datasource_stat'].\
fillna(duv_comb_final['TrueTemp_datasource_syn'])

duv_comb_final.head()

,Well ID,DST Start Depth (MD) (m),DST End Depth (MD) (m),DST Bottom Hole Temp. (degC),DST Test Date,Test Type,DST Misrun,DST Number,Formation DSTd,Unnamed: 9,...,diff_depth_abs,True Temperature (oC),TrueTemp_datasource_syn,Well_ID,Depth (ft),Temp (degC),Field,TrueTemp_datasource_stat,TempC_Fin,TrueTemp_datasource
0,100010107020W500,1410.6,1423.4,48.33,26025,DST,N,2,TRmontney,NaN,...,4.0,60.97,synthetic,NaN,NaN,NaN,NaN,NaN,60.97,synthetic
1,100010608109W500,1686.0,1694.0,34.26,33647,DST,N,1,Dmuskeg,NaN,...,4.4,63.98,synthetic,NaN,NaN,NaN,NaN,NaN,63.98,synthetic
2,100010904012W500,3698.0,3718.0,104.40,30798,LRT,N,1,Melkton,NaN,...,NaN,NaN,synthetic,NaN,NaN,NaN,NaN,NaN,NaN,synthetic
3,100011206021W400,822.0,825.0,29.00,29543,DST,N,4,Dwintrbrn,NaN,...,8.1,39.07,synthetic,NaN,NaN,NaN,NaN,NaN,39.07,synthetic
4,100011506915W500,1801.0,1818.0,64.00,30760,DST,N,4,Dwabamun,NaN,...,3.6,69.47,synthetic,NaN,NaN,NaN,NaN,NaN,69.47,synthetic


# 4 Combine Eaglebine and Duvernay temperature datasets

In [288]:
Duvernay = duv_comb_final[['UWI', 'DST Bottom Hole Temp. (degC)', 'TempC_Fin','Depth_SS(m)']]
Duvernay = Duvernay.rename(columns={'DST Bottom Hole Temp. (degC)': 'BHT', 'TempC_Fin': 'TrueTemp', 
                                    'Depth_SS(m)':"SSTVD(m)"})
Duvernay['Field'] = 'Duvernay'

Eaglebine = egb_comb_final[['UWI', 'TempC_BHT', 'TempC_Fin', 'BHT_below sea level (ft)']]
Eaglebine = Eaglebine.rename(columns={'TempC_BHT': 'BHT', 'TempC_Fin': 'TrueTemp',
                                      'BHT_below sea level (ft)':'SSTVD(m)'})
Eaglebine['Field'] = 'Eaglebine'

tt_basins = pd.concat((Duvernay, Eaglebine))
tt_basins.head()

,UWI,BHT,TrueTemp,SSTVD(m),Field
0,100010107020W500,48.33,60.97,716.0,Duvernay
1,100010608109W500,34.26,63.98,1044.4,Duvernay
2,100010904012W500,104.40,NaN,2424.2,Duvernay
3,100011206021W400,29.00,39.07,171.9,Duvernay
4,100011506915W500,64.00,69.47,936.4,Duvernay


In [289]:
tt_basins.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836 entries, 0 to 323
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   UWI       836 non-null    object 
 1   BHT       836 non-null    float64
 2   TrueTemp  615 non-null    float64
 3   SSTVD(m)  836 non-null    float64
 4   Field     836 non-null    object 
dtypes: float64(3), object(2)
memory usage: 39.2+ KB


In [290]:
len(tt_basins.UWI.unique())

836

In [291]:
print(tt_basins.Field.value_counts())

Duvernay     512
Eaglebine    324
Name: Field, dtype: int64


# 5 Load well headers data from both basins

In [292]:
egb_wh = pd.read_excel(eaglebine_dir / 'Eaglebine well headers SPE April 21 2021.xlsx')
duv_wh = pd.read_excel(duvernay_dir / 'Duvernay well headers SPE April 21 2021 .xlsx')

## 5.1 Load X, Y locations into geopandas of eaglebine basin

In [293]:
lat_nad83_col = "SurfaceLatitude_NAD83"
lon_nad83_col = "SurfaceLongitude_NAD83"
lat_wgs84_col = "WGS84Latitude"
lon_wgs84_col = "WGS84Longitude"
# Useful cols to rename
td_col = "td"
kb_col = "elevation"
uwi_col = "UWI"
# Useful cols
useful_cols = [td_col, "Elevation", "displayapi"]

wh_nad83 = egb_wh.dropna(subset=[lat_nad83_col, lon_nad83_col])
wh_wgs84 = egb_wh.dropna(subset=[lat_wgs84_col, lon_wgs84_col])

geometry_nad83 = gpd.points_from_xy(wh_nad83[lon_nad83_col], wh_nad83[lat_nad83_col])
wh_nad83.drop([lat_nad83_col, lon_nad83_col], axis=1, inplace=True)
geometry_wgs84 = gpd.points_from_xy(wh_wgs84[lon_wgs84_col], wh_wgs84[lat_wgs84_col])
wh_wgs84.drop([lat_wgs84_col, lon_wgs84_col], axis=1, inplace=True)

gdf_wh_1 = gpd.GeoDataFrame(wh_nad83[useful_cols],
                            crs="epsg:4269",
                            geometry=geometry_nad83)
print(gdf_wh_1.info())
gdf_wh_2 = gpd.GeoDataFrame(wh_wgs84[useful_cols],
                            crs="epsg:4326",
                            geometry=geometry_wgs84)
print(gdf_wh_2.info())
# Convert epsg 4326 to 4269
gdf_wh_egb = gdf_wh_1.append(gdf_wh_2.to_crs(epsg=4269))
print(gdf_wh_egb.info())
gdf_wh_egb.rename(columns={"Elevation": kb_col, "displayapi": uwi_col}, inplace=True)
display(gdf_wh_egb.head())

C:\Users\fredd\miniconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 308 entries, 0 to 323
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   td          308 non-null    int64   
 1   Elevation   308 non-null    float64 
 2   displayapi  308 non-null    int64   
 3   geometry    308 non-null    geometry
dtypes: float64(1), geometry(1), int64(2)
memory usage: 12.0 KB
None
<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 16 entries, 18 to 315
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   td          16 non-null     int64   
 1   Elevation   16 non-null     float64 
 2   displayapi  16 non-null     int64   
 3   geometry    16 non-null     geometry
dtypes: float64(1), geometry(1), int64(2)
memory usage: 640.0 bytes
None
<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 324 entries, 0 to 315
Data columns (total 4 columns):
 #   Column      N

,td,elevation,UWI,geometry
0,19635,338.68,42013301410000,POINT (-98.46982 28.69010)
1,10775,384.59,42013301930000,POINT (-98.20840 28.78004)
2,6175,442.00,42013302760000,POINT (-98.42277 28.99481)
3,8198,392.00,42013305480000,POINT (-98.29596 28.96339)
4,11000,493.23,42013310190000,POINT (-98.15547 28.81794)


## 5.2 Load X, Y locations into geopandas of Duvernay basin

In [294]:
# Useful cols
td_col = "TD"
uwi_col = "UWI"
kb_col = "KB"
useful_cols = ["TD meters ", "Elevation Meters", "UWI "]
lon_col = "SurfaceLongitude_NAD83"
lat_col = "SurfaceLatitude_NAD83"
geometry = gpd.points_from_xy(duv_wh[lon_col], duv_wh[lat_col])
duv_wh.drop([lat_col, lon_col], axis=1, inplace=True)

gdf_wh_duv = gpd.GeoDataFrame(duv_wh[useful_cols],
                          crs="epsg:4269",
                          geometry=geometry)
gdf_wh_duv.rename(columns={"UWI ": uwi_col,
                       "TD meters ": td_col,
                       "Elevation Meters": kb_col}, inplace=True)
display(gdf_wh_duv.head())

,TD,KB,UWI,geometry
0,2545.0,555.7,100072507623W500,POINT (-117.42380 55.61364)
1,2430.0,560.9,100062007823W500,POINT (-117.53767 55.77087)
2,2515.5,558.4,100052107723W500,POINT (-117.51617 55.68372)
3,2411.0,631.4,100061307518W500,POINT (-116.65361 55.49460)
4,2408.0,627.0,100070907618W500,POINT (-116.72624 55.56697)


## 5.3 Concat Well Headers datasets from both Basins

In [295]:
gdf_wh_egb['UWI'] = gdf_wh_egb['UWI'].astype('str')

In [296]:
# Rename columns of gdf_wh_egb
gdf_wh_egb.rename(columns={'td':'TD', 'elevation':'KB'}, inplace=True)

In [297]:
# Concat boht datasets
gdf_wh_com = gdf_wh_egb.append(gdf_wh_duv)

In [298]:
gdf_wh_com.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 836 entries, 0 to 511
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   TD        836 non-null    float64 
 1   KB        836 non-null    float64 
 2   UWI       836 non-null    object  
 3   geometry  836 non-null    geometry
dtypes: float64(2), geometry(1), object(1)
memory usage: 32.7+ KB


# 6. Create Dataset for Machine Learning

In [299]:
df_tt = gpd.GeoDataFrame(tt_basins.merge(gdf_wh_com, on='UWI'))
df_tt.head()

,UWI,BHT,TrueTemp,SSTVD(m),Field,TD,KB,geometry
0,100010107020W500,48.33,60.97,716.0,Duvernay,2767.6748,701.093481,POINT (-116.90355 55.02798)
1,100010608109W500,34.26,63.98,1044.4,Duvernay,1745.0000,645.600000,POINT (-115.39055 55.98751)
2,100010904012W500,104.40,NaN,2424.2,Duvernay,4817.0000,1283.800000,POINT (-115.65743 52.42418)
3,100011206021W400,29.00,39.07,171.9,Duvernay,1492.6000,651.600000,POINT (-113.00267 54.16741)
4,100011506915W500,64.00,69.47,936.4,Duvernay,2710.0000,873.100000,POINT (-116.18816 54.96904)


In [300]:
df_tt.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 836 entries, 0 to 835
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   UWI       836 non-null    object  
 1   BHT       836 non-null    float64 
 2   TrueTemp  615 non-null    float64 
 3   SSTVD(m)  836 non-null    float64 
 4   Field     836 non-null    object  
 5   TD        836 non-null    float64 
 6   KB        836 non-null    float64 
 7   geometry  836 non-null    geometry
dtypes: float64(5), geometry(1), object(2)
memory usage: 58.8+ KB


In [301]:
print(len(df_tt.UWI.unique()))
print(df_tt.Field.value_counts())

836
Duvernay     512
Eaglebine    324
Name: Field, dtype: int64


## 6.1 Transform to NAD27 Zone 11 for Duvernay Basin and to NAD 27 (Texas Central Zone, USA) for Eaglebine Basin

In [302]:
# Extract information form eaglebine basin
df_tt_egb = df_tt.loc[df_tt.Field == 'Eaglebine']
df_tt_egb.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 324 entries, 512 to 835
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   UWI       324 non-null    object  
 1   BHT       324 non-null    float64 
 2   TrueTemp  243 non-null    float64 
 3   SSTVD(m)  324 non-null    float64 
 4   Field     324 non-null    object  
 5   TD        324 non-null    float64 
 6   KB        324 non-null    float64 
 7   geometry  324 non-null    geometry
dtypes: float64(5), geometry(1), object(2)
memory usage: 22.8+ KB


In [303]:
# Transform this dataset to epsg=32039
df_tt_egb = df_tt_egb.to_crs(epsg=32039)
df_tt_egb.head()

,UWI,BHT,TrueTemp,SSTVD(m),Field,TD,KB,geometry
512,42013301410000,61.666667,141.555556,3725.418000,Eaglebine,19635.0,338.68,POINT (2597996.267 -350331.933)
513,42013301930000,107.222222,132.778000,3168.700800,Eaglebine,10775.0,384.59,POINT (2681233.966 -316108.676)
514,42013302760000,65.555556,89.161111,1747.418400,Eaglebine,6175.0,442.00,POINT (2611185.942 -239228.351)
515,42013305480000,76.666667,107.961111,2379.268800,Eaglebine,8198.0,392.00,POINT (2651947.655 -249935.738)
516,42013310190000,98.888889,133.722222,3203.289504,Eaglebine,11000.0,493.23,POINT (2697925.581 -301994.814)


In [304]:
# Extract information form duvernay basin
df_tt_duv = df_tt.loc[df_tt.Field == 'Duvernay']
df_tt_duv.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 512 entries, 0 to 511
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   UWI       512 non-null    object  
 1   BHT       512 non-null    float64 
 2   TrueTemp  372 non-null    float64 
 3   SSTVD(m)  512 non-null    float64 
 4   Field     512 non-null    object  
 5   TD        512 non-null    float64 
 6   KB        512 non-null    float64 
 7   geometry  512 non-null    geometry
dtypes: float64(5), geometry(1), object(2)
memory usage: 36.0+ KB


In [305]:
# Transform this dataset to epsg=26711
df_tt_duv = df_tt_duv.to_crs(epsg=26711)
df_tt_duv.head()

,UWI,BHT,TrueTemp,SSTVD(m),Field,TD,KB,geometry
0,100010107020W500,48.33,60.97,716.0,Duvernay,2767.6748,701.093481,POINT (506233.853 6097692.519)
1,100010608109W500,34.26,63.98,1044.4,Duvernay,1745.0000,645.600000,POINT (600473.368 6205641.486)
2,100010904012W500,104.40,NaN,2424.2,Duvernay,4817.0000,1283.800000,POINT (591360.466 5808850.167)
3,100011206021W400,29.00,39.07,171.9,Duvernay,1492.6000,651.600000,POINT (760971.701 6009315.438)
4,100011506915W500,64.00,69.47,936.4,Duvernay,2710.0000,873.100000,POINT (552040.176 6091430.565)


In [306]:
# Merge both dataframes
df_tt_comb = df_tt_egb.append(df_tt_duv)
df_tt_comb.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 836 entries, 512 to 511
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   UWI       836 non-null    object  
 1   BHT       836 non-null    float64 
 2   TrueTemp  615 non-null    float64 
 3   SSTVD(m)  836 non-null    float64 
 4   Field     836 non-null    object  
 5   TD        836 non-null    float64 
 6   KB        836 non-null    float64 
 7   geometry  836 non-null    geometry
dtypes: float64(5), geometry(1), object(2)
memory usage: 58.8+ KB


In [307]:
df_tt_comb.head()

,UWI,BHT,TrueTemp,SSTVD(m),Field,TD,KB,geometry
512,42013301410000,61.666667,141.555556,3725.418000,Eaglebine,19635.0,338.68,POINT (2597996.267 -350331.933)
513,42013301930000,107.222222,132.778000,3168.700800,Eaglebine,10775.0,384.59,POINT (2681233.966 -316108.676)
514,42013302760000,65.555556,89.161111,1747.418400,Eaglebine,6175.0,442.00,POINT (2611185.942 -239228.351)
515,42013305480000,76.666667,107.961111,2379.268800,Eaglebine,8198.0,392.00,POINT (2651947.655 -249935.738)
516,42013310190000,98.888889,133.722222,3203.289504,Eaglebine,11000.0,493.23,POINT (2697925.581 -301994.814)


## 6.2 Transform to x and Y columns the geometry

In [308]:
x_col, y_col = "X", "Y"

In [309]:
df_tt_comb[x_col] = df_tt_comb["geometry"].x
df_tt_comb[y_col] = df_tt_comb["geometry"].y

In [310]:
df_tt_comb.drop(["geometry", 'TD', 'KB'], axis=1, inplace=True)
df_tt_comb.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 836 entries, 512 to 511
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   UWI       836 non-null    object 
 1   BHT       836 non-null    float64
 2   TrueTemp  615 non-null    float64
 3   SSTVD(m)  836 non-null    float64
 4   Field     836 non-null    object 
 5   X         836 non-null    float64
 6   Y         836 non-null    float64
dtypes: float64(5), object(2)
memory usage: 52.2+ KB


# 7. Load Contest Data for Prediction

In [311]:
df_contest = pd.read_csv(ROOT_DIR / "gtx/data/set_assign.csv")
df_contest.head()

,UWI,Set
0,42021301990000,Validation_Testing
1,100102606420W500,Validation_Testing
2,100141705519W500,Validation_Testing
3,100141503621W400,Validation_Testing
4,100043406718W500,Validation_Testing


In [312]:
df_contest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 821 entries, 0 to 820
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   UWI     821 non-null    object
 1   Set     821 non-null    object
dtypes: object(2)
memory usage: 13.0+ KB


## 7.1 Split data into training and validation/testing sets

In [315]:
data_basins = df_tt_comb.merge(df_contest, on='UWI')
data_basins.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 821 entries, 0 to 820
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   UWI       821 non-null    object 
 1   BHT       821 non-null    float64
 2   TrueTemp  615 non-null    float64
 3   SSTVD(m)  821 non-null    float64
 4   Field     821 non-null    object 
 5   X         821 non-null    float64
 6   Y         821 non-null    float64
 7   Set       821 non-null    object 
dtypes: float64(5), object(3)
memory usage: 57.7+ KB


In [321]:
data_basins.Field.value_counts()

Duvernay     499
Eaglebine    322
Name: Field, dtype: int64

In [328]:
# Data for training the ML Algorithms
training = data_basins['Set'] == 'Training'
data_training = data_basins[training]
data_training.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 615 entries, 0 to 820
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   UWI       615 non-null    object 
 1   BHT       615 non-null    float64
 2   TrueTemp  615 non-null    float64
 3   SSTVD(m)  615 non-null    float64
 4   Field     615 non-null    object 
 5   X         615 non-null    float64
 6   Y         615 non-null    float64
 7   Set       615 non-null    object 
dtypes: float64(5), object(3)
memory usage: 43.2+ KB


In [329]:
validation = data_basins['Set'] == 'Validation_Testing'
data_val = data_basins[validation]
data_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206 entries, 12 to 818
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   UWI       206 non-null    object 
 1   BHT       206 non-null    float64
 2   TrueTemp  0 non-null      float64
 3   SSTVD(m)  206 non-null    float64
 4   Field     206 non-null    object 
 5   X         206 non-null    float64
 6   Y         206 non-null    float64
 7   Set       206 non-null    object 
dtypes: float64(5), object(3)
memory usage: 14.5+ KB


In [330]:
data_val.drop(columns='TrueTemp', axis=1, inplace=True)

C:\Users\fredd\miniconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
